In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv


In [23]:
class BatsmanState(TypedDict):
    runs: int 
    balls: int 
    fours: int 
    sixes: int 

    sr: float 
    bpb:float 
    bp: float 
    summary: str

In [32]:
def calc_sr(state: BatsmanState):

    sr = (state['runs']/state['balls'])*100 
    return {'sr': sr}  

In [33]:
def calc_bpb(state: BatsmanState):
    bpb = state['balls']/(state['fours']+state['sixes']) 

    return {'bpb': bpb}

In [34]:
def calc_bp(state: BatsmanState):

    bp = ((state['fours']*4 + state['sixes']*6)/state['runs'])*100 
    return {'bp': bp}

In [35]:
def summary(state:BatsmanState):

    summary = f"strikerate - {state['sr']} \n balls per boundary - {state['bpb']} , Boundary percent - {state['bp']}"
    return {'summary':summary}

In [37]:
graph = StateGraph(BatsmanState)

graph.add_node('calc_sr',calc_sr) 
graph.add_node('calc_bpb',calc_bpb) 
graph.add_node('calc_bp',calc_bp) 
graph.add_node('summary',summary) 

graph.add_edge(START,'calc_sr')
graph.add_edge(START,'calc_bpb') 
graph.add_edge(START,'calc_bp')

graph.add_edge('calc_sr','summary')
graph.add_edge('calc_bpb','summary') 
graph.add_edge('calc_bp','summary')

graph.add_edge('summary',END)

app = graph.compile() 


In [38]:
initial_state = {
    'runs' : 100,
    'balls' : 50,
    'fours' : 6,
    'sixes' : 4
}

final_state = app.invoke(initial_state)
print(final_state)

{'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4, 'sr': 200.0, 'bpb': 5.0, 'bp': 48.0, 'summary': 'strikerate - 200.0 \n balls per boundary - 5.0 , Boundary percent - 48.0'}
